In [ ]:
# 使用正负无穷
inf = float('inf') #正无穷
ninf = float('-inf') #负无穷

# 浮动回望期权相当于用回望期的最优价格代替普通期权的行权价
def get_float_lookback_price(s0, r, q, sigma, T, option_type):
    """
    定义函数计算回望期权的价值
    param s0: 标的资产当前价格
    param r: 无风险收益率
    param q: 基础资产分红收益率
    param sigma：标的资产年化波动率
    param T: 以年计算的期权到期时间
    param option_type: 'call'或'put'
    return 返回期权权益金
    """
    N_simulations = 1000
    n_steps = 200
    dt = T / n_steps
    
    opt_sum = 0
    
    # 模拟 N_simulations 条价格路径：
    for i in range(N_simulations):
        s = s0           # 每次模拟标的资产价格都从当前价格开始演化                                     
        max_price = float('-inf')    # 价格路径中的最大和最小价格
        min_price = float('inf')
        
        # 模拟标的资产价格的几何布朗运动，确定不同价格路径下的最大值和最小值
        for j in range(n_steps):
            z = gauss(0,1)
            # 资产价格的几何布朗运动
            s *= e ** ((r - 0.5*(sigma**2))*dt + sigma*sqrt(dt)*z)
            # 记录价格路径中的价格极值
            max_price = max(max_price, s)
            min_price = min(min_price, s)
        
        # 计算一条价格路径对应的回望期权的价值
        if option_type.lower()[0] == 'c':
            opt_sum += get_vanilla_price(s0, min_price, r, q, sigma, T, option_type)
        else:
            opt_sum += get_vanilla_price(s0, max_price, r, q, sigma, T, option_type)
    
    return opt_sum/N_simulations    # 对所有价格路径下得到的期权价值取均值即得到回望期权的当前的价值

get_float_lookback_price(s0, r, q, sigma, T, 'p')

# 固定回望期权相当于用回望期的最优值代替标的资产的终值计算收益的期权
def get_fixed_lookback_price(s0, X, r, q, sigma, T, option_type):
    """
    定义函数计算回望期权的价值
    param s0: 标的资产当前价格
    param X: 期权行权价格
    param r: 无风险收益率
    param q: 基础资产分红收益率
    param sigma：标的资产年化波动率
    param T: 以年计算的期权到期时间
    param option_type: 'call'或'put'
    return 返回期权权益金
    """
    N_simulations = 1000
    n_steps = 200
    dt = T / n_steps
    
    value_sum = 0
    
    # 模拟 N_simulations 条价格路径：
    for i in range(N_simulations):
        s = s0                       # 每次模拟标的资产价格都从当前价格开始演化   
        max_price = float('-inf')    # 价格路径中的最大和最小价格
        min_price = float('inf')

        # 模拟标的资产价格的几何布朗运动，确定不同价格路径下障碍条件是否会触及
        for j in range(n_steps):
            z = gauss(0,1)
            # 资产价格的几何布朗运动
            s *= e ** ((r - 0.5*(sigma**2))*dt + sigma*sqrt(dt)*z)
            # 记录价格路径中的价格极值
            max_price = max(max_price, s)
            min_price = min(min_price, s)
        
        # 计算一条价格路径对应的回望期权的价值
        if option_type.lower()[0] == 'c':
            value_sum += max((max_price - X), 0)
        else:
            value_sum += max((X - min_price), 0)
    
    return (e **(- r * T)) * value_sum / N_simulations    # 对所有价格路径下得到的期权价值取均值得到回望期权价值 

get_fixed_lookback_price(s0, strike, r, q, sigma, T, 'p')